# AdaGrad

Glorot条件：
- Xavier初始化
- BatchNormlization

将学习率衰减加入优化算法：
1. 线性衰减至固定的最小值
2. 指数衰减
3. 每次验证集误差提升时，按2～10之间的乘数对学习率进行衰减

AdaGrad：使用每个维度权重梯度的大小来自适应调整学习率
（Adaptive Subgradiwnt Methods 自适应次梯度方法）


- gt所有样本梯度的平均数$g_t = \frac{\partial{L}}{\partial{w}}=\frac{1}{n}\sum_{i=1}^{n}\nabla_w{L(x^{(i)},y^{(i)},w)}$
- Gt过往迭代中gt的平方和
- 步长 $\frac{\eta}{\sqrt{G_t+\epsilon}}$ 梯度为分母，梯度大则步长小，梯度小则步长大
- $w_t = w_{(t-1)}-\frac{\eta}{\sqrt{diag(G_t)+\epsilon*I}} \odot g_t$

AdaGrad 非常适合于稀疏矩阵（低频特征、占比少、目标范围小）。
致命问题：迭代后期的Gt大、学习率非常小，容易出现梯度消失现象

torch.optim.Adagrad

# RMSprop
均方根传播（Root Mean Square Propogation）
继承AdaGrad几乎全部设置
- $g_t = \frac{\partial{L}}{\partial{w}}$
- $G_t = \alpha G_{t-1} + (1-\alpha)g_t^2$ 小于AdaGrad的Gt
- $\Delta{t} = B\Delta_{t-1} - \frac{\eta}{\sqrt{G_t}+\epsilon}* g_t$
- $w_t = w_{t-1} + \Delta{t}$

动量法的迭代公式：
- $V_t = \gamma V_{t-1} - \eta \frac{L}{\partial{w}}$
- $w_t = w_{t-1} + v_t$

torch.optim.RMSprop

pytorch在RMSprop中有参数centered,对梯度gt中心化使得梯度迭代更加平稳，其流程可能如下：
$$g_t = g_t-Var(g_t)$$
或者(更可能)
$$g_t = \frac{g_t}{Var(g_t)}$$

- 在陌生架构上尝试优化算法时，RMSprop首选。
- 缺点：让学习率变大（Gt可能小于$G_{t-1}$）的特性违反了优化算法中的“正定性”$0.95G_{t-1} + 0.05g_t^2 = G_{t-1} - 0.05G_{t-1} + 0.05g_t^2 = G_{t-1} + 0.05(g_t^2 - G_{t-1})$

# Adam 亚当法
迭代流程：
- $g_t = \frac{\partial{L}}{\partial{w}}$
- $V_t = \beta_1 V_{t-1} + (1-\beta_1)g_t$
- $G_t = \beta_2G_{t-1} + (1-\beta_2)g_t^2$
- $w_t = w_{t-1} - \frac{\eta}{\sqrt{G_t}+\epsilon} * V_{t}$

“偏差修正”(bias correction)
- $\hat{V_t} = \frac{V_t}{1-\beta_1} = \frac{\beta_1V_{t-1} + (1-\beta_1)g_t}{1-\beta_1} = \frac{\beta_1}{1-\beta_1}V_{t-1}+g_t$
- $\hat{G_t} = \frac{G_t}{1-\beta_2} = \frac{\beta_2G_{t-1} + (1-\beta_2)g_t^2}{1-\beta_2} = \frac{\beta_2}{1-\beta_2}G_{t-1}+g_t^2$

$$w_t = w_{t-1} - \frac{\eta}{\sqrt{\hat{G_t}+\epsilon}} * \hat{V_t}$$

torch.optim.Adam

超参数 amsgrad: True为改进后的亚当，默认False

AMSGrad 迭代流程：
- $g_t = \frac{\partial{L}}{\partial{w}}$
- $V_t = \beta_1 V_{t-1} + (1-\beta_1)g_t$
- $G_t = \beta_2G_{t-1} + (1-\beta_2)g_t^2$
- $\hat{G_t} = max(\hat{G_{t-1}},G_t)$ 每次使用历史最大Gt
- $w_t = w_{t-1} - \frac{\eta}{\sqrt{G_t}+\epsilon} * V_{t}$

实际使用中Adam和AMSGrad效果差不多

## 权重衰减 weight_decay
处理正则化、防止过拟合的方式

在w进行迭代时，以某种方式直接施加在w上的一个衰减项，衰减的程度一般由超参数$\lambda$控制

以普通梯度下降为例，带有权重衰减的w的迭代公式为：
$$w_t = (1-\lambda)w_{t-1} - \eta * g_t$$

很多时候把对损失函数的L2正则化与权重衰减等价。但这个性质对任何除了`梯度下降`外的算法都不适用，如不适用于AdaGrad、RMSprop、Adam

在Adam上施加的权重衰减表现如下：
- $g_t=\frac{\partial{L}}{\partial{w}} + \lambda w$
- $w_t = w_{t-1}-\eta(\frac{V-t}{\sqrt{\hat{G_t}+\epsilon}}+\lambda w_{t-1})$

在设置权重衰减时，首先设置比较小的值，效果不好时，放弃这种正则化解决过拟合的方式选择其他方式

## 优化算法选择

《A Comparison of Optimization Algorithms for Deep Learning》

SGD - Nestrov 可以了解下